# Amazon Fine Food Reviews Analysis


Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews <br>

The dataset consists of reviews of fine foods from Amazon products.<br>

Number of reviews: 568,454<br>
Number of users: 256,059<br>
Number of products: 74,258<br>
Timespan: Oct 1999 - Oct 2012<br>
Number of Attributes/Columns in data: 10 

Attribute Information:

1. Id
2. ProductId - unique identifier for the product
3. UserId - unqiue identifier for the user
4. ProfileName
5. HelpfulnessNumerator - number of users who found the review helpful
6. HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
7. Score - rating between 1 and 5
8. Time - timestamp for the review
9. Summary - brief summary of the review
10. Text - text of the review


#### Objective:
Given a review, determine whether the review is positive (Rating of 4 or 5) or negative (rating of 1 or 2).

<br>
[Q] How to determine if a review is positive or negative?<br>
<br> 
[Ans] We could use the Score/Rating. A rating of 4 or 5 could be cosnidered a positive review. A review of 1 or 2 could be considered negative. A review of 3 is nuetral and ignored. This is an approximate way of determining the polarity (positivity/negativity) of a review.


In [1]:
%matplotlib inline
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

filtering only positive and negative reviews i.e. not taking into consideration those reviews with Score=3

In [2]:
# using the SQLite Table to read data.
con = sqlite3.connect('database.sqlite') 

filtered_data = pd.read_sql_query(""" 
                                    SELECT * FROM Reviews WHERE Score != 3
                                  """, con)

In [3]:
filtered_data.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
# Give reviews with Score>3 a positive rating (1), and reviews with a score<3 a negative rating (0).
def partition(x):
    if x < 3:
        return 0
    return 1
#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

Number of data points in our data (525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


Lets find some data anamolies like:
* Check if there are any duplicate reviews given by a user

In [12]:
display = pd.read_sql_query("""
SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*)
FROM Reviews
WHERE Score != 3
GROUP BY UserId, ProductId, ProfileName, Time, Score, Text
HAVING COUNT(*)>1
""", con)

print(display.shape)
display.head()

(734, 7)


,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
0,A10GBEDF4RCOVI,B0037272UY,Superchiller,1310774400,1,"My girlfriend and I have been rescuing, foster...",2
1,A10NJPL8SA82WC,B0026RQTGE,STANIMAL,1343779200,5,These things are awesome. These Greenies real...,2
2,A10NJPL8SA82WC,B002QWHJOU,STANIMAL,1343779200,5,These things are awesome. These Greenies real...,2
3,A10NJPL8SA82WC,B002QWP89S,STANIMAL,1343779200,5,These things are awesome. These Greenies real...,2
4,A10NJPL8SA82WC,B002QWP8H0,STANIMAL,1343779200,5,These things are awesome. These Greenies real...,2


In [17]:
display.head(100)

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
0,A10GBEDF4RCOVI,B0037272UY,Superchiller,1310774400,1,"My girlfriend and I have been rescuing, foster...",2
1,A10NJPL8SA82WC,B0026RQTGE,STANIMAL,1343779200,5,These things are awesome. These Greenies real...,2
2,A10NJPL8SA82WC,B002QWHJOU,STANIMAL,1343779200,5,These things are awesome. These Greenies real...,2
3,A10NJPL8SA82WC,B002QWP89S,STANIMAL,1343779200,5,These things are awesome. These Greenies real...,2
4,A10NJPL8SA82WC,B002QWP8H0,STANIMAL,1343779200,5,These things are awesome. These Greenies real...,2
...,...,...,...,...,...,...,...
95,A1TMAVN4CEM8U8,B0034KN29O,Gunner,1332115200,5,Diamond Almonds<br />Almonds are a good source...,2
96,A1TMAVN4CEM8U8,B0034KP00S,Gunner,1332115200,5,Diamond Almonds<br />Almonds are a good source...,2
97,A1TMAVN4CEM8U8,B0049Z5OSK,Gunner,1332115200,5,Diamond Almonds<br />Almonds are a good source...,2
98,A1TMAVN4CEM8U8,B0049Z9ANU,Gunner,1332115200,5,Diamond Almonds<br />Almonds are a good source...,2


In [18]:
sample1 = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3
and UserId = "A10NJPL8SA82WC"
""", con)
print(sample1.shape)

(8, 10)


In [19]:
sample1.head(10)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,21114,B002QWP89S,A10NJPL8SA82WC,STANIMAL,0,0,5,1343779200,Awesome Product,These things are awesome. These Greenies real...
1,21115,B002QWP89S,A10NJPL8SA82WC,STANIMAL,0,0,5,1343779200,Awesome Product,These things are awesome. These Greenies real...
2,74763,B0026RQTGE,A10NJPL8SA82WC,STANIMAL,0,0,5,1343779200,Awesome Product,These things are awesome. These Greenies real...
3,74764,B0026RQTGE,A10NJPL8SA82WC,STANIMAL,0,0,5,1343779200,Awesome Product,These things are awesome. These Greenies real...
4,329544,B002QWHJOU,A10NJPL8SA82WC,STANIMAL,0,0,5,1343779200,Awesome Product,These things are awesome. These Greenies real...
5,329545,B002QWHJOU,A10NJPL8SA82WC,STANIMAL,0,0,5,1343779200,Awesome Product,These things are awesome. These Greenies real...
6,355635,B002QWP8H0,A10NJPL8SA82WC,STANIMAL,0,0,5,1343779200,Awesome Product,These things are awesome. These Greenies real...
7,355636,B002QWP8H0,A10NJPL8SA82WC,STANIMAL,0,0,5,1343779200,Awesome Product,These things are awesome. These Greenies real...


In [20]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

### Drop duplicates

In [21]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(364173, 10)

In [22]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

69.25890143662969

Found rows where HelpfulnessNumerator is greater than HelpfulnessDenominator. These records are not valid, hence dropping them

In [24]:
final[final.HelpfulnessNumerator>final.HelpfulnessDenominator]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
59301,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,1,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
41159,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,1,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [25]:
final = final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [26]:
#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(364171, 10)


1    307061
0     57110
Name: Score, dtype: int64

# Text Preprocessing.

In the Preprocessing phase we do the following in the order below:-

* Begin by removing the html tags
* Remove any punctuations or limited set of special characters like , or . or # etc.
* Check if the word is made up of english letters and is not alpha-numeric
* Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
* Convert the word to lowercase
* Remove Stopwords
* Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)
* After which we collect the words used to describe positive and negative reviews